In [ ]:
# sphinx ignore

import sys

sys.path.append("../..")

%config Completer.use_jedi = False

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from gpytorch.kernels import MaternKernel, ScaleKernel
from sklearn.preprocessing import StandardScaler

from vanguard.datasets.bike import BikeDataset
from vanguard.models import InducingPointKernelGPModel
from vanguard.uncertainty import GaussianUncertaintyGPController
from vanguard.vanilla import GaussianGPController
from vanguard.warps import SetWarp, warpfunctions

In [ ]:
DATASET = BikeDataset(seed=42)

In [ ]:
plt.hist(DATASET.train_y)
plt.xlabel("$y$", fontsize=15)
plt.show()

In [ ]:
scaler = StandardScaler()
scaled_train_x = scaler.fit_transform(DATASET.train_x)
scaled_test_x = scaler.transform(DATASET.test_x)

In [ ]:
class SparseGaussianGPController(GaussianGPController):
    gp_model_class = InducingPointKernelGPModel

In [ ]:
N_INDUCING_POINTS = 50
NUM_ITERS = int(len(scaled_train_x) / 64) * 10

In [ ]:
class ScaledMaternKernel(ScaleKernel):
    """A scaled matern kernel."""

    def __init__(self):
        super().__init__(MaternKernel(nu=1.5, ard_num_dims=scaled_train_x.shape[1]))

In [ ]:
gp = SparseGaussianGPController(
    train_x=scaled_train_x,
    train_y=DATASET.train_y,
    kernel_class=ScaledMaternKernel,
    y_std=DATASET.train_y_std,
    gp_kwargs={"n_inducing_points": N_INDUCING_POINTS},
    optim_kwargs={"lr": 0.01},
)

gp.fit(n_sgd_iters=NUM_ITERS)

posterior = gp.posterior_over_point(scaled_test_x)
mean, lower, upper = posterior.confidence_interval()
print(f"RMSE: {np.sqrt(np.mean((DATASET.test_y - mean) ** 2))}")
plt.errorbar(DATASET.test_y, mean, yerr=np.vstack([mean - lower, upper - mean]), marker="o", label="mean", linestyle="")
plt.xlabel("true y values")
plt.ylabel("predicted y values")
plt.legend()
plt.show()

In [ ]:
warp = warpfunctions.AffineWarpFunction() @ warpfunctions.BoxCoxWarpFunction(lambda_=0)


@SetWarp(warp_function=warp, ignore_methods=("fit", "__init__"))
class WarpedGaussianGPController(SparseGaussianGPController):
    pass

In [ ]:
gp = WarpedGaussianGPController(
    train_x=scaled_train_x,
    train_y=DATASET.train_y,
    kernel_class=ScaledMaternKernel,
    y_std=DATASET.train_y_std,
    gp_kwargs={"n_inducing_points": N_INDUCING_POINTS},
    optim_kwargs={"lr": 0.01},
)

gp.fit(n_sgd_iters=NUM_ITERS)

posterior = gp.posterior_over_point(scaled_test_x)
warp_mean, warp_lower, warp_upper = posterior.confidence_interval()
print(f"RMSE: {np.sqrt(np.mean((DATASET.test_y - warp_mean) ** 2))}")
plt.errorbar(
    DATASET.test_y,
    mean,
    yerr=np.vstack([warp_mean - warp_lower, warp_upper - warp_mean]),
    marker="o",
    label="mean",
    linestyle="",
)
plt.xlabel("true y values")
plt.ylabel("predicted y values")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
small_indices = DATASET.test_y < 0.5
plt.errorbar(
    DATASET.test_y[small_indices],
    mean[small_indices],
    yerr=np.vstack([mean - lower, upper - mean])[:, small_indices],
    marker="o",
    label="mean",
    linestyle="",
)
plt.title(f"No warping. RMSE: " f"{np.sqrt(np.mean((DATASET.test_y[small_indices] - mean[small_indices]) ** 2)):.4}")
plt.xlabel("true y values")
plt.ylabel("predicted y values")

plt.subplot(1, 2, 2)
small_indices = DATASET.test_y < 0.5
y_err = np.vstack([warp_mean - warp_lower, warp_upper - warp_mean])[:, small_indices]
plt.errorbar(
    DATASET.test_y[small_indices], warp_mean[small_indices], yerr=y_err, marker="o", label="mean", linestyle=""
)
plt.title(
    f"Affine-log warping. RMSE: "
    f"{np.sqrt(np.mean((DATASET.test_y[small_indices] - warp_mean[small_indices]) ** 2)):.4}"
)
plt.xlabel("true y values")

plt.tight_layout()
plt.show()

In [ ]:
warp = warpfunctions.AffineWarpFunction() @ warpfunctions.BoxCoxWarpFunction(lambda_=0)


@SetWarp(warp_function=warp, ignore_all=True)
class WarpedGaussianUncertaintyGPController(GaussianUncertaintyGPController):
    pass

In [ ]:
gp = WarpedGaussianUncertaintyGPController(
    train_x=scaled_train_x[:500],
    train_x_std=0.1,
    train_y=DATASET.train_y[:500],
    kernel_class=ScaledMaternKernel,
    y_std=0.001 * np.mean(np.abs(DATASET.train_y)),
    likelihood_kwargs={"learn_additional_noise": True},
    batch_size=None,
)
gp.fit(n_sgd_iters=NUM_ITERS)

posterior = gp.posterior_over_point(scaled_test_x)
mean, lower, upper = posterior.confidence_interval()
print(f"RMSE: {np.sqrt(np.mean((DATASET.test_y - mean) ** 2))}")
plt.errorbar(DATASET.test_y, mean, yerr=np.vstack([mean - lower, upper - mean]), marker="o", label="mean", linestyle="")
plt.xlabel("true y values")
plt.ylabel("predicted y values")
plt.legend()
plt.show()